In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
df_fea = pd.read_csv('fea.csv')
df_target = pd.read_csv('target.csv')

In [3]:
print("Структура данных")
print(df_fea.info())
print("Стастистика")
print(df_fea.describe())

Структура данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Device_ID                         5000 non-null   object 
 1   device_age_months                 5000 non-null   int64  
 2   battery_capacity_mah              5000 non-null   int64  
 3   avg_screen_on_hours_per_day       5000 non-null   float64
 4   avg_charging_cycles_per_week      5000 non-null   float64
 5   avg_battery_temp_celsius          5000 non-null   float64
 6   fast_charging_usage_percent       5000 non-null   float64
 7   overnight_charging_freq_per_week  5000 non-null   int64  
 8   gaming_hours_per_week             5000 non-null   float64
 9   video_streaming_hours_per_week    5000 non-null   float64
 10  background_app_usage_level        5000 non-null   object 
 11  signal_strength_avg               5000 non-null   ob

In [4]:
print("Стркуктура данных")
print(df_target.info())
print("Статистика")
print(df_target.describe())

Стркуктура данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Device_ID                       5000 non-null   object 
 1   current_battery_health_percent  5000 non-null   float64
 2   recommended_action              5000 non-null   object 
dtypes: float64(1), object(2)
memory usage: 117.3+ KB
None
Статистика
       current_battery_health_percent
count                     5000.000000
mean                        62.595600
std                         17.723528
min                         10.000000
25%                         50.200000
50%                         64.100000
75%                         76.900000
max                        100.000000


In [5]:
df = pd.merge(df_fea, df_target, on='Device_ID')

In [6]:
X_raw = df[['device_age_months', 'battery_capacity_mah', 'current_battery_health_percent']]
y_raw = df['recommended_action']

In [7]:
le = LabelEncoder()
y_encoded = le.fit_transform(y_raw)
num_classes = len(le.classes_)

In [8]:
sc = StandardScaler()
X_scaled = sc.fit_transform(X_raw)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [10]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long) 
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

In [11]:
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=32, shuffle=True)

In [12]:
class BatteryClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(BatteryClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(p=0.3), 
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.2), 
            nn.Linear(64, num_classes)
        )
    def forward(self, x):
        return self.network(x)

In [13]:
model = BatteryClassifier(input_dim=3, num_classes=num_classes)

In [14]:
criterion = nn.CrossEntropyLoss()

In [15]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [16]:
epochs = 60 
for epoch in range(epochs):
    model.train() 
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad() 
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Эпоха {epoch+1}, Loss: {loss.item():.4f}")

Эпоха 10, Loss: 0.0559
Эпоха 20, Loss: 0.0524
Эпоха 30, Loss: 0.1469
Эпоха 40, Loss: 0.0273
Эпоха 50, Loss: 0.0211
Эпоха 60, Loss: 0.0011


In [17]:
model.eval()

BatteryClassifier(
  (network): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [69]:
with torch.no_grad(): 
    outputs = model(X_test_t)
    test_loss = criterion(outputs, y_test_t)
    preds = torch.argmax(outputs, dim=1)
    acc = (preds == y_test_t).float().mean()
    print(f" Loss: {test_loss.item():.4f}")
    print(f"Accurac: {acc:.4f}")

 Loss: 0.0141
Accurac: 0.9970
